In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import csv

In [3]:
data_nase = pd.read_csv('out.c-cleaning.ALL_POSITIONS_NEW.csv', encoding='utf-8', low_memory=False)
data_Honza = pd.read_csv('out.c-cleaning.ALL_HONZA_NEW.csv', encoding='utf-8', low_memory=False)
data_wkend = pd.read_csv('out.c-cleaning.ALL_HONZA_NEW_WKEND.csv', encoding='utf-8', low_memory=False)

data = pd.concat([data_nase, data_Honza, data_wkend], ignore_index = True)
data = data.dropna(axis=0,subset=['DELAY'],how='any')
data.head()

,UNICORN,TRIP_ID,START_TIME,HEADSIGN,DATE,BEARING,DELAY,DELAY_STOP_ARRIVAL,DELAY_STOP_DEPARTURE,IS_CANCELED,...,LAST_STOP_SEQUENCE,NEXT_STOP_ARRIVAL_TIME,NEXT_STOP_DEPARTURE_TIME,NEXT_STOP_ID,NEXT_STOP_SEQUENCE,ORIGIN_TIMESTAMP,SHAPE_DIST_TRAVELED,SPEED,LATITUDE,LONGITUDE
0,333_1052_200302_15_May,333_1052_200302,07:54:00,"Praha,Kačerov",15 May,24,184.0,NaN,182.0,False,...,25.0,"Fri, 15 May 2020 08:35:00 Z","Fri, 15 May 2020 08:35:00 Z",U228Z5,26.0,"Fri, 15 May 2020 08:36:30 Z",17.7,1,50.03789,14.45764
1,333_1052_200302_15_May,333_1052_200302,07:54:00,"Praha,Kačerov",15 May,18,199.0,NaN,182.0,False,...,25.0,"Fri, 15 May 2020 08:35:00 Z","Fri, 15 May 2020 08:35:00 Z",U228Z5,26.0,"Fri, 15 May 2020 08:37:38 Z",18.0,35,50.03988,14.45823
2,333_1052_200302_15_May,333_1052_200302,07:54:00,"Praha,Kačerov",15 May,42,201.0,195.0,NaN,False,...,25.0,"Fri, 15 May 2020 08:35:00 Z","Fri, 15 May 2020 08:35:00 Z",U228Z5,26.0,"Fri, 15 May 2020 08:38:15 Z",18.2,16,50.04186,14.46014
3,333_1052_200302_15_May,333_1052_200302,07:54:00,"Praha,Kačerov",15 May,28,173.0,NaN,182.0,False,...,25.0,"Fri, 15 May 2020 08:35:00 Z","Fri, 15 May 2020 08:35:00 Z",U228Z5,26.0,"Fri, 15 May 2020 08:36:19 Z",17.7,19,50.03770,14.45743
4,333_1052_200302_15_May,333_1052_200302,07:54:00,"Praha,Kačerov",15 May,25,219.0,NaN,182.0,False,...,25.0,"Fri, 15 May 2020 08:35:00 Z","Fri, 15 May 2020 08:35:00 Z",U228Z5,26.0,"Fri, 15 May 2020 08:37:05 Z",17.7,12,50.03805,14.45760


In [4]:
soubor = open('bus_stops_333.json', encoding='utf-8')
data_stops = json.load(soubor)
soubor.close()

In [26]:
info_bus_stops = pd.json_normalize(data_stops)
#info_bus_stops.head()

In [6]:
info_stops_clean = info_bus_stops.drop(columns=['arrival_time_seconds','departure_time_seconds','drop_off_type','pickup_type','stop_headsign',
                                                'stop.properties.stop_code','stop.properties.stop_desc','stop.properties.stop_timezone','stop.properties.stop_url',
                                                'stop.properties.wheelchair_boarding','stop.properties.zone_id','stop.type','stop.geometry.type','stop.properties.level_id','stop.properties.location_type',
                                                'stop.properties.parent_station','stop.properties.platform_code','timepoint'])
info_stops_clean.head()

,arrival_time,departure_time,shape_dist_traveled,stop_id,stop_sequence,trip_id,stop.geometry.coordinates,stop.properties.stop_id,stop.properties.stop_name
0,10:32:00,10:32:00,0.00000,U1503Z2,1,333_566_200409,"[14.45708, 49.96319]",U1503Z2,"Dolní Břežany,Obecní úřad"
1,10:33:00,10:33:00,0.15836,U1502Z2,2,333_566_200409,"[14.45889, 49.96373]",U1502Z2,"Dolní Břežany,Náměstí"
2,10:35:00,10:35:00,0.77992,U1501Z2,3,333_566_200409,"[14.46199, 49.96894]",U1501Z2,"Dolní Břežany,Na kopečku"
3,10:39:00,10:39:00,3.53721,U545Z2,4,333_566_200409,"[14.46853, 49.99163]",U545Z2,Písnice
4,10:40:00,10:40:00,3.91690,U546Z2,5,333_566_200409,"[14.46768, 49.99496]",U546Z2,Ke Březině


In [19]:
unicorn_try = data[data['UNICORN'] == '333_1052_200302_06_May'].sort_values(by='SHAPE_DIST_TRAVELED')
len(unicorn_try)
#unicorn_try

97

In [20]:
stop_dist_333 = info_stops_clean['shape_dist_traveled']

In [21]:
bus_dist_333 = unicorn_try['SHAPE_DIST_TRAVELED']

In [24]:
i = 0
for distance in stop_dist_333:
    
    nejmensivzdalenost = 10000

    for position in bus_dist_333:

        #tatonejmensivzdalenost = abs(stop_time['shape_dist_traveled'] - position['shape_dist_traveled'])
        tatonejmensivzdalenost = abs(distance - position)
        #print(tatonejmensivzdalenost)
        
        if tatonejmensivzdalenost < nejmensivzdalenost:
            
            nejmensivzdalenost = tatonejmensivzdalenost
            #print(nejmensivzdalenost)
            
            info_stops_clean['nejblizsi_pozice'][i] = position
    i += 1

C:\ProgramData\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
info_stops_clean

,arrival_time,departure_time,shape_dist_traveled,stop_id,stop_sequence,trip_id,stop.geometry.coordinates,stop.properties.stop_id,stop.properties.stop_name,nejblizsi_pozice
0,10:32:00,10:32:00,0.00000,U1503Z2,1,333_566_200409,"[14.45708, 49.96319]",U1503Z2,"Dolní Břežany,Obecní úřad",0.0
1,10:33:00,10:33:00,0.15836,U1502Z2,2,333_566_200409,"[14.45889, 49.96373]",U1502Z2,"Dolní Břežany,Náměstí",0.0
2,10:35:00,10:35:00,0.77992,U1501Z2,3,333_566_200409,"[14.46199, 49.96894]",U1501Z2,"Dolní Břežany,Na kopečku",0.9
3,10:39:00,10:39:00,3.53721,U545Z2,4,333_566_200409,"[14.46853, 49.99163]",U545Z2,Písnice,3.4
4,10:40:00,10:40:00,3.91690,U546Z2,5,333_566_200409,"[14.46768, 49.99496]",U546Z2,Ke Březině,4.1
5,10:41:00,10:41:00,4.11683,U871Z2,6,333_566_200409,"[14.46765, 49.99676]",U871Z2,Lipovická,4.1
6,10:42:00,10:42:00,4.61837,U369Z2,7,333_566_200409,"[14.4666, 50.0012]",U369Z2,Sídliště Písnice,4.7
7,10:43:00,10:43:00,5.09108,U783Z2,8,333_566_200409,"[14.46417, 50.00515]",U783Z2,U Libušské sokolovny,5.1
8,10:44:00,10:44:00,5.46109,U335Z2,9,333_566_200409,"[14.46275, 50.00834]",U335Z2,Libuš,5.3
9,10:45:00,10:45:00,5.77175,U804Z2,10,333_566_200409,"[14.46015, 50.01056]",U804Z2,U Zvoničky,5.8


In [ ]:
# i = 0
# for distance in stop_dist_333:
    
#     nejmensivzdalenost = 10000

#     for index, position in bus_dist_333.iterrows():
        
#         #print(position['SHAPE_DIST_TRAVELED'], position['DELAY_STOP_DEPARTURE'])
    
#         tatonejmensivzdalenost = abs(distance - position['SHAPE_DIST_TRAVELED'])
        
#         #print(position['SHAPE_DIST_TRAVELED'])
        
#         if tatonejmensivzdalenost < nejmensivzdalenost:
            
#             nejmensivzdalenost = tatonejmensivzdalenost
            
#             info_stops_clean['NEJBLIZSI_POZICE'][i] = position['SHAPE_DIST_TRAVELED']
#             info_stops_clean['DELAY_STOP_DEPARTURE'][i] = position['DELAY_STOP_DEPARTURE']
            
#     i += 1